# TEI Conference 2019 #

## Etape 3 : L'Analyse ##

### Préambule ###

L'objectif présent est d'analyser les abstracts afin de les regrouper en cluster, permettant de dégager des thématiques, de comparer les différentes conférences ou bien d'étudier si les distinctions faites par les organisateurs de la conférence 2019 entre les différentes interventions est bien pertinente.
Nous pourrons également critiquer les choix de l'ordinateur.

Nous allons faire ici de la classification non supervisée grâce au package SKLearn, librairie dédiée au Machine Learning notamment.

### Les packages ### 

Il faut lancer la cellule ci-dessous une seule fois afin de télécharger la librairie sklearn. Cela doit être fait une seule fois, au premier lancement. Ensuite, il ne faudra plus jamais le lancer. Il est possible qu'il faille redémarrer le noyau après.

In [52]:
pip install sklearn
pip install nltk

SyntaxError: invalid syntax (<ipython-input-52-c99697bebcaa>, line 1)

Il faut lancer la cellule ci-dessous à chaque lancement de ce notebook-ci.

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import nltk
from nltk.stem import PorterStemmer
import os


Tout d'abord, il faut ' stemmer ' les abstracts, c'est à dire qu'il faut prendre chaque mot de chaque abstract et le remplacer par son lemme. Cela permettra à la machine de ne pas être faussée par l'usage divers d'un seul et même mot racine (les mots "buy", "buys", "bought" et "buying" ne deviennent plus qu'un seul et même mot). Ainsi, l'analyse n'en sera que plus précise et plus juste.

Ce texte stemmé est stocké dans le cache, dans un répertoire propre.

In [60]:
Path = "./cache2019/cacheTXT/" #chemin permettant d'accès aux abstracts
Path_output = "./cache2019/cacheSTEM/" #chemin de sortie des abstracts stemmés
filelist = os.listdir(Path) #filelist est une liste regroupant tous les chemins vers les différents abstracts.
if not os.path.exists("./cache2019/cacheSTEM/"): #permet de créer un dossier dans le cache s'il est supprimé.
    os.makedirs("./cache2019/cacheSTEM/")

In [81]:
stemmer = PorterStemmer() #J'importe une méthode acceptant une chaîne de caractère à une variable.
documents = [] #j'instancie une liste qui regroupera l'ensemble des chemins vers les documents
for abstract in filelist:
    with open(Path + abstract, "r", encoding="UTF-8") as y:
        documents.append(Path_output + abstract)
        texte = y.read()
        liste_mots_tokenise = [] # j'instancie une liste qui accueillera chaque lemme (le lemme étant une str)
        liste_mots_a_tokenise = texte.split() #je crée une liste sur laquelle je vais pouvoir boucler
        for elem in liste_mots_a_tokenise:
            mot_tokenise = stemmer.stem(elem) #mot_tokenise est une str, et est le lemme du mot sur lequel je boucle
            liste_mots_tokenise.append(mot_tokenise) #j'ajoute chaque mot à une liste, qu'ensuite je join pour recréer l'abstract sous la forme d'une str
        resultat = ' '.join(liste_mots_tokenise)
        with open(Path_output + abstract, "w", encoding="UTF-8") as z:
            z.write(resultat) #resultat est une str
        



documents = vectorizer.transform(documents)
print(documents)


  (0, 4035)	0.8714423053310905
  (0, 2838)	0.333745611880824
  (0, 142)	0.35944704064650296
  (1, 4035)	0.6849882856854673
  (1, 2838)	0.26233731497627444
  (1, 148)	0.6181759640183353
  (1, 142)	0.2825396594369106
  (2, 4035)	0.6569841659774607
  (2, 2838)	0.251612276715042
  (2, 186)	0.6569841659774607
  (2, 142)	0.2709886962883761
  (3, 4035)	0.8714423053310905
  (3, 2838)	0.333745611880824
  (3, 142)	0.35944704064650296
  (4, 4035)	0.6849882856854673
  (4, 2838)	0.26233731497627444
  (4, 145)	0.6181759640183353
  (4, 142)	0.2825396594369106
  (5, 4035)	0.7051478776999994
  (5, 2838)	0.2700580502802617
  (5, 142)	0.29085496114527937
  (5, 115)	0.5875700057279885
  (6, 4035)	0.7208338508359468
  (6, 2838)	0.2760654757520075
  (6, 165)	0.561946838609136
  :	:
  (39, 170)	0.6569841659774607
  (39, 142)	0.2709886962883761
  (40, 4035)	0.6569841659774607
  (40, 2838)	0.251612276715042
  (40, 142)	0.2709886962883761
  (40, 47)	0.6569841659774607
  (41, 4035)	0.7051478776999994
  (41, 2838

Ici, on associe à chaque mot de tous les abstracts une coordonnée unique à chaque mot utilisé, pour ensuite pouvoir les placer sur un plan en 2D et ainsi relever des ressemblances.

In [77]:
#vectorize the text i.e. convert the strings to numeric features
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(documents)
print(documents)

['use machin learn for the autom classif of stage direct in tei-encod drama corpora author daria maximova (nation research univers higher school of economics, moscow, ru) frank fischer (dariah-eu and nation research univers higher school of economics, moscow, ru) abstract the <stage> tag is a core element for the encod of drama. the tei guidelin suggest nine valu for it type attribute, which is wide use in larg corpora such as the french théâtre classique, the shakespear folger librari or the swedish dramawebben. thi paper introduc an approach to automat assign stage-direct type to the tei-p5-encod russian drama corpus, rusdracor (https://dracor.org/). the corpu current featur 144 play rang from mid-18th to mid-20th centuri which make for 32 753 stage direct with 144,525 tokens. We select 18 play compris 6,569 stage direct to repres the breadth of the corpus. for the manual annot we establish a clear set of rule to identifi the stage-direct type propos by the tei guidelin (https://www.

Ensuite, on réunit les termes vectorisés en cluster, c'est à dire en groupe de ressemblance.

In [78]:
#cluster documents, ici 10 clusters
true_k = 4
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=3, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

Enfin, on montre les termes que l'ordinateur a pu rassembler.

In [79]:
#print top terms per cluster clusters
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])

Top terms per cluster:
Cluster 0:
 text
 parliamentari
 graph
 stage
 texts
 tei
 markup
 indic
 tibetan
 direct
Cluster 1:
 tei
 data
 thi
 encod
 edit
 project
 text
 line
 digit
 xml
Cluster 2:
 tei
 paper
 text
 odd
 use
 thi
 encod
 format
 xml
 element
